# RNA e Deep Learning - Projeto final
### Engenharia e Análise de Dados (2025.1)

Marcel Pontes &emsp; &emsp;(mfnp2@cesar.school)


Rice Image Dataset:
https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset/data


# Carregamento do dateset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muratkokludataset/rice-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 219M/219M [00:03<00:00, 72.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/muratkokludataset/rice-image-dataset/versions/1


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

REDUCED_HEIGHT = 32
REDUCED_WIDTH = 32

transform = transforms.Compose([
    transforms.Resize((REDUCED_HEIGHT, REDUCED_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_dataset = datasets.ImageFolder(
    root=f"{path}/Rice_Image_Dataset/",
    transform=transform
)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

data_loader_kwargs = {"shuffle": True, "num_workers": 2, "batch_size":32}
if use_cuda:
  cuda_kwargs = {
      "pin_memory": True,
  }
  data_loader_kwargs.update(cuda_kwargs)

dataloader = DataLoader(image_dataset, **data_loader_kwargs)

In [4]:
for images, labels in dataloader:
  print("Shape das imagens do batch: ", images.shape)
  print("Labels:", labels)
  break  # parar depois de um batch

Shape das imagens do batch:  torch.Size([32, 3, 32, 32])
Labels: tensor([0, 0, 2, 2, 1, 4, 4, 2, 3, 2, 4, 1, 3, 1, 4, 3, 0, 2, 3, 3, 2, 4, 2, 4,
        4, 1, 2, 0, 2, 0, 4, 2])


# Definição do modelo de rede

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, output_size),
        )

    def forward(self, x):
        x = self.dense(x)
        output = F.log_softmax(x, dim=1)
        return output

input_size = 32 * 32 * 3  # height x width x channels

model = NeuralNet(input_size, 5)

# Treinamento do modelo

In [14]:
import torch.optim as optim

lr = 0.001
optimizer = optim.AdamW(model.parameters(), lr=lr)

* Criação do objeto de treinamento

In [20]:
def train(data_loader: DataLoader, model: nn.Module, optimizer_obj, runtime_device, epoch: int):
  model.train()
  batch_size_sum = 0
  for batch_idx, (data, target) in enumerate(data_loader):
    data, target = data.to(runtime_device), target.to(runtime_device)

    optimizer_obj.zero_grad()
    output = model(data)

    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer_obj.step()

    batch_size_sum += len(data)
    if (batch_idx + 1) % 293 == 0:
      print(f"Epoch: {epoch+1} [{batch_size_sum}/{len(data_loader.dataset)}" \
            f"\tLoss: {loss.item():.4f}]")


In [21]:
epochs = 3
model = NeuralNet(input_size, 5).to(device)

for epoch in range(epochs):
  train(dataloader, model, optimizer, device, epoch)
  print()

Epoch: 1 [9376/75000	Loss: 1.6822]
Epoch: 1 [18752/75000	Loss: 1.6717]
Epoch: 1 [28128/75000	Loss: 1.6753]
Epoch: 1 [37504/75000	Loss: 1.6575]
Epoch: 1 [46880/75000	Loss: 1.6980]
Epoch: 1 [56256/75000	Loss: 1.6622]
Epoch: 1 [65632/75000	Loss: 1.6290]
Epoch: 1 [75000/75000	Loss: 1.6367]

Epoch: 2 [9376/75000	Loss: 1.6540]
Epoch: 2 [18752/75000	Loss: 1.6377]
Epoch: 2 [28128/75000	Loss: 1.6174]
Epoch: 2 [37504/75000	Loss: 1.6275]
Epoch: 2 [46880/75000	Loss: 1.6616]
Epoch: 2 [56256/75000	Loss: 1.6447]
Epoch: 2 [65632/75000	Loss: 1.6478]
Epoch: 2 [75000/75000	Loss: 1.6309]

Epoch: 3 [9376/75000	Loss: 1.6143]
Epoch: 3 [18752/75000	Loss: 1.6034]
Epoch: 3 [28128/75000	Loss: 1.5689]
Epoch: 3 [37504/75000	Loss: 1.6698]
Epoch: 3 [46880/75000	Loss: 1.6571]
Epoch: 3 [56256/75000	Loss: 1.6741]
Epoch: 3 [65632/75000	Loss: 1.6454]
Epoch: 3 [75000/75000	Loss: 1.5947]



# Avaliação

In [23]:
model.eval()
correct = 0
for data, target in dataloader:
  data, target = data.to(device), target.to(device)
  prediction = model(data)
  pred = prediction.argmax(dim=1, keepdim=True)
  correct += pred.eq(target.view_as(pred)).sum().item()

print("A acurácia de classificação com todos os dados ao final do treinamento")
print("Accuracy: ", 100*correct/len(dataloader.dataset) )

A acurácia de classificação com todos os dados ao final do treinamento
Accuracy:  15.994666666666667


# Conclusão

Apesar das diversas tentativas de trocas de otimizador da função de perda, do valor de _learning rate_ e incluindo até mesmo uma redução de dimensionalidade do dataset, de forma a se ter uma rede neural menos pesada, o modelo treinado teve extrema dificuldade de se ajustar às imagens do dataset, de forma que, com uma acurácia de 16% e uma perda que não se reduzia com o passar das épocas de treinamento, foi observado um underfit.

Ao pesquisar alternativas para melhorar a classificação dos dados, foi sugerida a redução da resolução das imagens. Logo, aplicou-se um `transforms.Resize` para dimensões de 32 x 32 saindo das 250 x 250 originais no processo de carregamento do dataset e consequentemente, a quantidade de nós de entrada do dataset também foi reduzida para refletir a mudança.

Uma solução frequentemente comum para lidar com problemas de convergência é a variação do _learning rate_ do modelo treinado. No entanto, essa opção também se mostrou infetiva.

Além de uma longa verificação das imagens de entrada e do seu processo de carregamento, experimentou-se a minimização do custo com os algoritmos de Gradiente Descendente Estocástico (SGD) e de Adaptive Movement Estimation (Adam) e três de suas variações. Porém, o resultado desejado não foi atingido.

Apesar da não convergência, o estudo foi interessante no entendimento das etapas de criação e arquitetura de uma rede neural para resolução de um desafio de classificação de imagens de grandes proporções. A estratégia adotada de criar o modelo com base em conhecimento passados e das aulas da respectiva disciplina além de pesquisas online proporcionaram uma vantajosa experiência de aprendizado.


<br>
_Obs.: O número de épocas de treinamento foi mantido baixo para que não fosse gasto muito tempo nas tentativas de otimização além de estar sendo verificado que as iterações já não estavam contribuindo para uma melhora do desempenho._

